In [1]:
import pandas as pd

df = pd.DataFrame()

In [2]:
import os

In [3]:
list_file = list(os.walk('./part'))[0][2]
if '.DS_Store' in list_file:
    list_file.remove('.DS_Store')
if '0.null.txt' in list_file:
    list_file.remove('0.null.txt')
    
list_file.sort()
    
len(list_file)

100

In [4]:
from bs4 import BeautifulSoup
from lxml import etree

In [ ]:
a = 7

with open('./part/' + list_file[a], 'r', encoding='UTF-8') as file:
    resp = file.read()
    
soup = BeautifulSoup(resp, 'lxml')
html = etree.HTML(str(soup).encode('ascii', 'xmlcharrefreplace').decode('utf-8'))

with open('./part.html', 'w', encoding='UTF-8') as file:
    file.write(str(soup))

print(soup.prettify())

In [6]:
title = html.xpath('//h1[@id="title"]/span/text()')[0].strip()

title

'AC Compressor Clutch KIT Coil Pulley FITS 2014-2016 Subaru XV CROSSTREK 2.0L, 1495'

In [7]:
price = html.xpath('//span[@class="a-offscreen"]/text()')[1].strip()

price

'$49.99'

In [8]:
about = ''
list_about = html.xpath('//ul[@class="a-unordered-list a-vertical a-spacing-mini"]/li/span/text()')
for i in range(len(list_about)):
    about += str(i+1) + '. ' + list_about[i].strip() + '\n'

print(about)

1. Warranty Period: 3 Months Warranty
2. Manufacturer Part Number: 1495
3. Warranty: 90 Day
4. Brand: CoolTech



In [9]:
src = html.xpath('//div[@id="imgTagWrapperId"]/img/@src')[0].strip()

src

'https://m.media-amazon.com/images/I/417JJgYzZgL.__AC_SY300_SX300_QL70_ML2_.jpg'

In [10]:
list_tr = html.xpath('//table[@class="a-normal a-spacing-micro"]/tr')

list_tr

[]

In [11]:
list_tr_2 = html.xpath('//table[@id="productDetails_techSpec_section_1"]/tr')

list_tr_2

[<Element tr at 0x129021200>,
 <Element tr at 0x1290212c0>,
 <Element tr at 0x129021280>,
 <Element tr at 0x129021300>,
 <Element tr at 0x129021340>,
 <Element tr at 0x1290213c0>,
 <Element tr at 0x129021400>,
 <Element tr at 0x129021440>]

In [12]:
list_tr_3 = html.xpath('//table[@id="productDetails_detailBullets_sections1"]/tr')

list_tr_3

[<Element tr at 0x12666be40>,
 <Element tr at 0x110058ec0>,
 <Element tr at 0x129020e40>,
 <Element tr at 0x129021f00>]

In [13]:
df_temp = pd.DataFrame([{'序号': int(list_file[a].replace('.txt', '')),
                         'ASIN': '',
                         'Title': title,
                         'Price': price,
                         'About': about.strip(),
                         'Pic': '',
                         'Url': '',
                         'Src': src}])

for tr in list_tr:
    df_temp.loc[0, tr.xpath('./td/span/text()')[0].strip().upper()] = tr.xpath('./td/span/text()')[1].strip()
    
for tr in list_tr_2:
    df_temp.loc[0, tr.xpath('./th/text()')[0].strip().lower()] = tr.xpath('./td/text()')[0].strip()

for tr in list_tr_3:
    if tr.xpath('./th/text()')[0].strip() == 'Best Sellers Rank':
        df_temp.loc[0, tr.xpath('./th/text()')[0].strip().upper()] = ' '.join(tr.xpath('./td/descendant::*/text()')).strip()
    else:
        df_temp.loc[0, tr.xpath('./th/text()')[0].strip().upper()] = ' '.join(tr.xpath('./td/text()')).strip()
    
df = pd.concat([df, df_temp], ignore_index=True).fillna('')
    
len(df)

1

In [14]:
df = df.drop_duplicates(ignore_index=True)
df = df.sort_values(by=['序号'], ascending=[True]).reset_index(drop=True)
df.to_excel('./part.xlsx', index=False)

df

,序号,ASIN,Title,Price,About,Pic,Url,Src,manufacturer,brand,model,item weight,package dimensions,item model number,exterior,manufacturer part number,CUSTOMER REVIEWS,BEST SELLERS RANK,DATE FIRST AVAILABLE
0,15,B09282P1H1,AC Compressor Clutch KIT Coil Pulley FITS 2014...,$49.99,1. Warranty Period: 3 Months Warranty\n2. Manu...,,,https://m.media-amazon.com/images/I/417JJgYzZg...,‎CoolTech,‎CoolTech,‎XV XV Crosstrek,‎4.2 pounds,‎7.25 x 7.25 x 3.25 inches,‎1495,‎Smooth,‎1495,5.0 out of 5 stars,"#438,966 in Automotive ( See Top 100 in Automo...","April 12, 2021"
